In [25]:
import numpy as np
from scapy.all import sniff
from scapy.layers.inet import IP
import time


# Número máximo de paquetes a capturar
num_paquetes = 10

# Listas para almacenar las características de los paquetes
times = []
src_ips = []
dst_ips = []
protocols = []
lengths = []


# Función para manejar cada paquete capturado
def packet_handler(packet):
    global times, src_ips, dst_ips, protocols, lengths
    if IP in packet:
        times.append(time.time())  # Registro de tiempo de llegada del paquete
        src_ips.append(packet[IP].src)
        dst_ips.append(packet[IP].dst)
        protocols.append(packet[IP].proto)
        lengths.append(len(packet))
        

        # Si se han capturado suficientes paquetes, detener la captura
        if len(src_ips) >= num_paquetes:
            return True  # Devuelve True para detener la captura

# Captura los paquetes y llama a packet_handler para cada uno
sniff(prn=packet_handler, store=0, count=num_paquetes)

# Organizar los datos en un numpy array
data = np.vstack((times, src_ips, dst_ips, protocols, lengths)).T

# Imprimir el array
print("Datos capturados:\n", data)

True
Datos capturados:
 [['1712227155.1922808' '192.168.100.33' '192.168.100.255' '17' '63']
 ['1712227156.4118726' '192.168.100.16' '255.255.255.255' '17' '214']
 ['1712227156.559964' '192.168.100.43' '239.255.255.250' '17' '216']
 ['1712227156.5751724' '192.168.100.43' '239.255.255.250' '17' '217']
 ['1712227156.876194' '192.168.100.33' '239.255.255.250' '17' '143']
 ['1712227156.9174612' '192.168.100.2' '13.69.109.130' '6' '171']
 ['1712227156.9696407' '192.168.100.2' '13.69.109.130' '6' '8652']
 ['1712227156.970163' '192.168.100.2' '1.1.1.1' '17' '92']
 ['1712227157.0034685' '1.1.1.1' '192.168.100.2' '17' '212']
 ['1712227157.0044684' '192.168.100.2' '20.189.173.5' '6' '66']]


In [30]:
import numpy as np
from scapy.all import sniff
from scapy.layers.inet import IP
import time


# Número máximo de paquetes a capturar
num_paquetes = 10

# Listas para almacenar las características de los paquetes
times = []
src_ips = []
dst_ips = []
protocols = []
lengths = []
numbers = []
strings = []

# Función para manejar cada paquete capturado
numb=0
def packet_handler(packet):
    global times, src_ips, dst_ips, protocols, lengths, strings, numbers, numb
    if IP in packet:
        times.append(time.time())  # Registro de tiempo de llegada del paquete
        src_ips.append(packet[IP].src)
        dst_ips.append(packet[IP].dst)
        protocols.append(packet[IP].proto)
        lengths.append(len(packet))
        strings.append(str(packet))

        numbers.append(numb)
        numb=numb+1
        

        # Si se han capturado suficientes paquetes, detener la captura
        if len(src_ips) >= num_paquetes:
            return True  # Devuelve True para detener la captura

# Captura los paquetes y llama a packet_handler para cada uno
sniff(prn=packet_handler, store=0, count=num_paquetes)

# Organizar los datos en un numpy array
data = np.vstack((times, src_ips, dst_ips, protocols, lengths, strings, numbers)).T

# Imprimir el array
print("Datos capturados:\n", data)


True
Datos capturados:
 [['1712227404.9600124' '192.168.100.2' '13.69.109.130' '6' '171'
  'Ether / IP / TCP 192.168.100.2:58731 > 13.69.109.130:https PA / Raw'
  '0']
 ['1712227405.1055362' '192.168.100.2' '13.69.109.130' '6' '6536'
  'Ether / IP / TCP 192.168.100.2:58731 > 13.69.109.130:https PA / Raw'
  '1']
 ['1712227405.1065366' '13.69.109.130' '192.168.100.2' '6' '60'
  'Ether / IP / TCP 13.69.109.130:https > 192.168.100.2:58731 A / Padding'
  '2']
 ['1712227405.1065366' '13.69.109.130' '192.168.100.2' '6' '148'
  'Ether / IP / TCP 13.69.109.130:https > 192.168.100.2:58731 PA / Raw'
  '3']
 ['1712227405.1075358' '192.168.100.2' '13.69.109.130' '6' '89'
  'Ether / IP / TCP 192.168.100.2:58731 > 13.69.109.130:https PA / Raw'
  '4']
 ['1712227405.267313' '192.168.100.33' '192.168.100.255' '17' '63'
  'Ether / IP / UDP 192.168.100.33:45253 > 192.168.100.255:32414 / Raw'
  '5']
 ['1712227405.3083112' '13.69.109.130' '192.168.100.2' '6' '60'
  'Ether / IP / TCP 13.69.109.130:https > 19

In [23]:
from scapy.all import sniff

strings=[]
# Función para manejar cada paquete capturado
def packet_handler(packet):
    # Obtener la representación en forma de cadena del paquete
    packet_str = str(packet)
    
    # Imprimir la cadena del paquete
    print(packet_str)
    strings.append(packet_str)

# Capturar los paquetes y llamar a packet_handler para cada uno
sniff(prn=packet_handler, count=5)


Ether / IP / UDP / DNS Qry "b'_oculusal_sp._tcp.local.'" 
Ether / IP / UDP / DNS Ans "b'flaco::DESKTOP-RMG725S._oculusal_sp._tcp.local.'" 
Ether / IP / UDP / DNS Qry "b'_oculusal_sp_v2._tcp.local.'" 
Ether / IP / UDP / DNS Ans "b'flaco::DESKTOP-RMG725S._oculusal_sp_v2._tcp.local.'" 
Ether / IP / TCP 192.168.100.2:58731 > 13.69.109.130:https PA / Raw


<Sniffed: TCP:1 UDP:4 ICMP:0 Other:0>

In [9]:
strings[1]

'Ether / IP / UDP / DNS Qry "b\'44595f8a-4ae5-f959-60d3-ad4350a40a11._googlezone._tcp.local.\'" '